In [ ]:
import numpy as np
import random
import pandas as pd
import pickle
from scipy.stats import sem
from sklearn.linear_model import LogisticRegression
import os
import matplotlib.pyplot as plt

# Set random seed for reproducibility
random_seed = 20250323
random.seed(random_seed)
np.random.seed(random_seed)

# General Processing Functions
def prepare_hist_matrix(a, R, n_back):
    # Prepare history matrix
    num_trials = len(a)
    uR = np.zeros(num_trials)
    uR[(R==0) & (a<3)] = 1

    # Now pads with zeros
    a_hist = np.zeros((num_trials, n_back))
    R_hist = np.zeros((num_trials, n_back))
    uR_hist = np.zeros((num_trials, n_back))

    a_cleaned = a.copy()+0.
    a_cleaned[a>2] = 0
    a_cleaned[a==2] = -1

    # fill history matrices with n_back information
    for nn in range(n_back):
        a_hist[(1+nn):,nn] = a_cleaned[0:num_trials-nn-1]
        R_hist[(1+nn):,nn] = R[0:num_trials-nn-1]
        uR_hist[(1+nn):,nn] = uR[0:num_trials-nn-1]

    return np.fliplr(a_hist), np.fliplr(R_hist), np.fliplr(uR_hist)

# Basic logistic Regression fitting functions
def logistic_prepare_predictors(a, R, n_back):
    num_trials = len(a)

    # Get history matrices
    a_hist, R_hist, _ = prepare_hist_matrix(a, R, n_back)

    # create and align component arrays
    past_r = np.multiply(a_hist, R_hist)
    past_ur = np.multiply(a_hist, R_hist==0)
    past_c = a_hist

    # identify choice trials
    choice_trials = (a==1)|(a==2)

    left_trials = (a==1)+0
    left_trials[a==2]=-1

    left_choice_trials = left_trials[choice_trials]

    # create output arrays: predictors and targets
    glm_mat = np.concatenate((past_r[choice_trials,:], past_ur[choice_trials,:]), axis=1)
    glm_target = left_choice_trials
    return glm_mat, glm_target

# Simulation loop
def simulation_session_loop(environment_properties, agent_properties, num_trials):
    # Initialize trial_history_dict
    trial_history_dict = {'Trial': [],
                         'l_prob': [],
                         'r_prob': [],
                         'lreward_available': [],
                         'rreward_available': [],
                         'Choice': [],
                         'Reward':[],
                         'p_left':[]}
    for tt in range(1, num_trials+1):
        trial_history_dict, lr = simulation_reward_assignment(trial_history_dict, tt, environment_properties)
        trial_history_dict, choice = simulation_agent(trial_history_dict, tt, agent_properties)

        # Decide choice
        temp_random = random.random()
        if temp_random < agent_properties['miss rate']:
            choice = 4  # Choice is Miss
        elif temp_random < (agent_properties['miss rate'] + agent_properties['alarm rate']):
            choice = 3  # Choice is Alarm

        trial_history_dict['Choice'][-1] = choice

        # Evaluate Choice
        if (choice==1) & (lr[0]==1):  # left choice & left reward, where lr[0] is the lreward_available
            trial_history_dict['Reward'][-1] = 1
        elif (choice==2) & (lr[1]==1):
            trial_history_dict['Reward'][-1] = 1
        else:
            trial_history_dict['Reward'][-1] = 0  # This should be redundant, but just to be safe

    trial_history_pd = pd.DataFrame(trial_history_dict)
    return trial_history_pd

# Environment Simulation
def simulation_reward_assignment(trial_history_dict, current_trial, environment_properties):
    # Define fixed parameters
    betaRC = environment_properties['betaRC']
    betaUC = environment_properties['betaUC']
    nback = environment_properties['nback']
    tau = environment_properties['tau']
    Pmax = environment_properties['Pmax']
    
    # Create exponential filter inside this function
    n = np.array(range(1, Num_Trials))
    exponential_filter = np.exp((1-n)/tau)

    if current_trial == 1:
        nomissalarm = 1
        lreward_available = 0
        rreward_available = 0
        l_prob = 0.5
        r_prob = 0.5
    else:
        a = np.array(trial_history_dict['Choice'])
        a_cleaned = a.copy()+0.
        a_cleaned[a>2] = 0
        a_cleaned[a==2] = -1
        R = np.array(trial_history_dict['Reward'])
        UR = 1-R
        RewC = a_cleaned*R
        UnrC = a_cleaned*UR

        if current_trial < nback+1:
            RewC_cleaned = RewC[::-1][0:current_trial-1]
            UnrC_cleaned = UnrC[::-1][0:current_trial-1]
            RC_weights = RewC_cleaned*exponential_filter[0:current_trial-1]
            UC_weights = UnrC_cleaned*(exponential_filter[0:current_trial-1])
        else:
            RewC_cleaned = RewC[::-1][0:nback]
            UnrC_cleaned = UnrC[::-1][0:nback]
            RC_weights = RewC_cleaned*exponential_filter[0:nback]
            UC_weights = UnrC_cleaned*(exponential_filter[0:nback])
        
        sum_weights = betaRC*np.sum(RC_weights) + betaUC*sum(UC_weights)
        l_prob = Pmax/(1+np.exp(-sum_weights))
        r_prob = Pmax-l_prob

        # Confirm a choice was made on the previous trial
        nomissalarm = (trial_history_dict['Choice'][-1]==1) | (trial_history_dict['Choice'][-1]==2)
        # Load reward baiting
        lreward_available = trial_history_dict['lreward_available'][-1]
        rreward_available = trial_history_dict['rreward_available'][-1]

        # If the previous choice was rewarded, clear reward
        if (trial_history_dict['Reward'][-1]==1) & (trial_history_dict['Choice'][-1]==1):
            lreward_available = 0
        elif (trial_history_dict['Reward'][-1]==1) & (trial_history_dict['Choice'][-1]==2):
            rreward_available = 0

    # Assign reward
    if nomissalarm == 1:
        rreward_available = (rreward_available==1) | (random.random() < r_prob)
        lreward_available = (lreward_available==1) | (random.random() < l_prob)

    rreward_available += 0
    lreward_available += 0

    # Update and append to trial_history_dict
    trial_history_dict = {'Trial': np.append(trial_history_dict['Trial'], current_trial),
                         'l_prob': np.append(trial_history_dict['l_prob'], l_prob),
                         'r_prob': np.append(trial_history_dict['r_prob'], r_prob),
                         'lreward_available': np.append(trial_history_dict['lreward_available'], lreward_available),
                         'rreward_available': np.append(trial_history_dict['rreward_available'], rreward_available),
                         'Choice': np.append(trial_history_dict['Choice'], 0),
                         'Reward': np.append(trial_history_dict['Reward'], 0),
                         'p_left': np.append(trial_history_dict['p_left'], 0.5)}  # is a place holder

    lr_reward_availibility = [lreward_available, rreward_available]
    return trial_history_dict, lr_reward_availibility

# Agent simulation
def simulation_agent(trial_history_dict, current_trial, agent_properties):
    # Simulation agent decision section
    if agent_properties['type'] == 'random':
        choice = random.randint(1, 2)
        p_lr = np.array([0.5, 0.5])
        if random.random() < p_lr[0]:
            choice = 1  # left choice
        else:
            choice = 2  # right choice

    if agent_properties['type'] == 'WinStay_LosesWitch':
        if current_trial == 1:
            p_lr = np.array([0.5, 0.5])
        else:
            a_previous = trial_history_dict['Choice'][current_trial-2]
            R_previous = trial_history_dict['Reward'][current_trial-2]
            if a_previous == 1:  # left choice trials
                if R_previous == 1:
                    p_lr = np.array([1, 0])
                else:
                    p_lr = np.array([0, 1])
            elif a_previous == 2:  # right choice trials
                if R_previous == 1:
                    p_lr = np.array([0, 1])
                else:
                    p_lr = np.array([1, 0])
            else:  # miss/alarm, stay
                print('include miss/alarm')

        if agent_properties['strategy'] == 'deterministic':
            if p_lr[0] >= 0.5:
                choice = 1  # left choice
            else:
                choice = 2  # right choice
        else:  # probabilistic
            if random.random() < p_lr[0]:
                choice = 1  # left choice
            else:
                choice = 2  # right choice

    if agent_properties['type'] == 'Foraging1B_GT':
        nback = 1
        agent_tau = 0.01
        agent_n = np.array(range(1, Num_Trials))
        agent_exponential_filter = np.exp((1-agent_n)/agent_tau)
        if current_trial == 1:
            p_lr = np.array([0.5, 0.5])
        else:
            a = np.array(trial_history_dict['Choice'][0:current_trial-1])
            a_cleaned = a.copy()+0.
            a_cleaned[a>2] = 0
            a_cleaned[a==2] = -1
            R = np.array(trial_history_dict['Reward'][0:current_trial-1])
            UR = 1-R
            RewC = a_cleaned*R
            UnrC = a_cleaned*UR

            if current_trial < nback+1:
                RewC_cleaned = RewC[::-1][0:current_trial-1]
                UnrC_cleaned = UnrC[::-1][0:current_trial-1]
                RC_weights = RewC_cleaned*agent_exponential_filter[0:current_trial-1]
                UC_weights = UnrC_cleaned*(agent_exponential_filter[0:current_trial-1])
            else:
                RewC_cleaned = RewC[::-1][0:nback]
                UnrC_cleaned = UnrC[::-1][0:nback]
                RC_weights = RewC_cleaned*agent_exponential_filter[0:nback]
                UC_weights = UnrC_cleaned*(agent_exponential_filter[0:nback])
            
            sum_weights = 1.5*np.sum(RC_weights) + (-6)*sum(UC_weights)
            l_prob = 1/(1+np.exp(-sum_weights))
            p_lr = np.array([l_prob, 1-l_prob])
            
        if agent_properties['strategy'] == 'deterministic':
            if p_lr[0] >= 0.5:
                choice = 1  # left choice
            else:
                choice = 2  # right choice
        else:  # probabilistic
            if random.random() < p_lr[0]:
                choice = 1  # left choice
            else:
                choice = 2  # right choice

    if agent_properties['type'] == 'Foraging20B_GT':
        nback = 20
        agent_tau = 20
        agent_n = np.array(range(1, Num_Trials))
        agent_exponential_filter = np.exp((1-agent_n)/agent_tau)
        if current_trial == 1:
            p_lr = np.array([0.5, 0.5])
        else:
            a = np.array(trial_history_dict['Choice'][0:current_trial-1])
            a_cleaned = a.copy()+0.
            a_cleaned[a>2] = 0
            a_cleaned[a==2] = -1
            R = np.array(trial_history_dict['Reward'][0:current_trial-1])
            UR = 1-R
            RewC = a_cleaned*R
            UnrC = a_cleaned*UR

            if current_trial < nback+1:
                RewC_cleaned = RewC[::-1][0:current_trial-1]
                UnrC_cleaned = UnrC[::-1][0:current_trial-1]
                RC_weights = RewC_cleaned*agent_exponential_filter[0:current_trial-1]
                UC_weights = UnrC_cleaned*(agent_exponential_filter[0:current_trial-1])
            else:
                RewC_cleaned = RewC[::-1][0:nback]
                UnrC_cleaned = UnrC[::-1][0:nback]
                RC_weights = RewC_cleaned*agent_exponential_filter[0:nback]
                UC_weights = UnrC_cleaned*(agent_exponential_filter[0:nback])
            
            sum_weights = 0.5*np.sum(RC_weights) + (-1)*sum(UC_weights)
            l_prob = 1/(1+np.exp(-sum_weights))
            p_lr = np.array([l_prob, 1-l_prob])
            
        if agent_properties['strategy'] == 'deterministic':
            if p_lr[0] >= 0.5:
                choice = 1  # left choice
            else:
                choice = 2  # right choice
        else:  # probabilistic
            if random.random() < p_lr[0]:
                choice = 1  # left choice
            else:
                choice = 2  # right choice

    if agent_properties['type'] == 'bias':
        const_bias = -0.7  # beta0, bias term in the RL model
        r = 2*random.random()-1
        if r+const_bias > 0:
            choice = 1
        else:
            choice = 2
        p_lr = np.array([0.5, 0.5])

    # Update trial history dictionary with decision parameters
    trial_history_dict['p_left'][-1] = p_lr[0]

    return trial_history_dict, choice

# Configuration
Environments = ['Long', 'Short']  # Run both environments
Simulation_Agent = 'Foraging1B_GT'  # Options: "WinStay_LosesWitch", "bias", "random", "Foraging1B_GT", "Foraging20B_GT"
Decision_Strategy = 'probabilistic'  # Options: "deterministic", "probabilistic"
Num_Trials = 500
Miss_Rate = 0
Alarm_Rate = 0
Num_Runs = 100  # Number of simulation runs for each agent type

# Set up environment properties
environment_properties_dict = {
    'Long': {'betaRC': 0.5,
             'betaUC': -1,
             'nback': 20,
             'tau': 20,
             'Pmax': 0.65},
    'Short': {'betaRC': 1.5,
              'betaUC': -6,
              'nback': 1,
              'tau': 0.01,
              'Pmax': 0.75}
}

# Function to run simulations for a given environment
def run_simulations_for_environment(env_name):
    print(f"\n{'='*50}")
    print(f"Starting simulations for {env_name} environment")
    print(f"{'='*50}")
    
    # Set up environment
    environment_properties = environment_properties_dict[env_name]
    n = np.array(range(1, Num_Trials))
    tau = environment_properties['tau']
    hyperbolic_filter = 1./(1+(n-1)/tau)
    exponential_filter = np.exp((1-n)/tau)
    n_back = environment_properties['nback']
    
    # Initialize dataframe for storing all simulations
    simulation_df = pd.DataFrame(columns=['Environment', 'Agent', 'a', 'R', 'L(prob)'])
    
    # Run simulations for Foraging1B_GT
    Simulation_Agent = 'Foraging1B_GT'
    agent_properties = {'type': Simulation_Agent,
                       'miss rate': Miss_Rate,
                       'alarm rate': Alarm_Rate,
                       'strategy': Decision_Strategy}
    
    print(f"\nRunning {Num_Runs} simulations for {Simulation_Agent}")
    for i in range(Num_Runs):
        trial_history = simulation_session_loop(environment_properties, agent_properties, Num_Trials)
        a = np.array(trial_history['Choice'])
        R = np.array(trial_history['Reward'])
        l_prob = trial_history['l_prob']
        
        temp_dict = {'Environment': env_name,
                    'Agent': Simulation_Agent,
                    'a': a, 
                    'R': R, 
                    'L(prob)': l_prob}
        
        simulation_df = pd.concat([simulation_df, pd.DataFrame([temp_dict])], ignore_index=True)
    
    # Run simulations for Foraging20B_GT
    Simulation_Agent = 'Foraging20B_GT'
    agent_properties = {'type': Simulation_Agent,
                       'miss rate': Miss_Rate,
                       'alarm rate': Alarm_Rate,
                       'strategy': Decision_Strategy}
    
    print(f"\nRunning {Num_Runs} simulations for {Simulation_Agent}")
    for i in range(Num_Runs):
        trial_history = simulation_session_loop(environment_properties, agent_properties, Num_Trials)
        a = np.array(trial_history['Choice'])
        R = np.array(trial_history['Reward'])
        l_prob = trial_history['l_prob']
        
        temp_dict = {'Environment': env_name,
                    'Agent': Simulation_Agent,
                    'a': a, 
                    'R': R, 
                    'L(prob)': l_prob}
        
        simulation_df = pd.concat([simulation_df, pd.DataFrame([temp_dict])], ignore_index=True)
    
    # Save the dataframe
#     save_directory = 'set your save directory here'
    os.makedirs(save_directory, exist_ok=True)
    
    output_filename = os.path.join(save_directory, f'simulation_data_{env_name}_combined_{random_seed}.pkl')
    pickle.dump(simulation_df, open(output_filename, 'wb'))
    print(f"\nSimulation data saved to {output_filename}")
    
    
    n_back = 20
    # Calculate coefficients
    print("Calculating regression coefficients...")
    coef_df_green = pd.DataFrame(columns=range(n_back*2))
    coef_df_yellow = pd.DataFrame(columns=range(n_back*2))
    intercept_all_green = []
    intercept_all_yellow = []
    
    for ss in range(len(simulation_df)):
        a = simulation_df.iloc[ss]['a']
        R = simulation_df.iloc[ss]['R']
        glm_mat, glm_target = logistic_prepare_predictors(a, R, 20)
        
        log_reg = LogisticRegression(solver='lbfgs', penalty='none',
                                    n_jobs=-1, random_state=0, multi_class='auto', 
                                    fit_intercept=True, max_iter=10000)
        
        log_reg.fit(glm_mat, glm_target)
        coef = log_reg.coef_[0]
        intercept = log_reg.intercept_[0]
        
        if simulation_df['Agent'][ss] == "Foraging1B_GT":
            coef_df_green.loc[ss, :] = coef
            intercept_all_green.append(intercept)
        elif simulation_df['Agent'][ss] == "Foraging20B_GT":
            coef_df_yellow.loc[ss, :] = coef
            intercept_all_yellow.append(intercept)
    
    # Save coefficients
    coef_filename = os.path.join(save_directory, f'coefficients_{env_name}_combined_{random_seed}.pkl')
    pickle.dump({'coef_df_green': coef_df_green, 'coef_df_yellow': coef_df_yellow, 
                'intercept_all_green': intercept_all_green, 'intercept_all_yellow': intercept_all_yellow}, 
                open(coef_filename, 'wb'))
    print(f"Regression coefficients calculated and saved to {coef_filename}")
    
    # Plotting
    print("Generating plots...")
    common_x = np.arange(-n_back, 0, 1)
    fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharey=False, sharex=True, tight_layout=True)
    
    # RewC
    ax[0].errorbar(common_x, coef_df_green.mean(axis=0)[:n_back],
                   yerr=[sem(coef_df_green.iloc[:, w].dropna()) for w in range(n_back)],
                   color='green', marker='o', markersize=3, linestyle='-', 
                   lw=2, capsize=4, label='Foraging1B_GT')
    ax[0].errorbar(common_x, coef_df_yellow.mean(axis=0)[:n_back],
                   yerr=[sem(coef_df_yellow.iloc[:, w].dropna()) for w in range(n_back)],
                   color='orange', marker='o', markersize=3, linestyle='-', 
                   lw=2, capsize=4, label='Foraging20B_GT')
    
    # UnrC
    ax[1].errorbar(common_x, coef_df_green.mean(axis=0)[n_back:2*n_back],
                   yerr=[sem(coef_df_green.iloc[:, w + n_back].dropna()) for w in range(n_back)],
                   color='green', marker='o', markersize=3, linestyle='-', 
                   lw=2, capsize=4, label='Foraging1B_GT')
    ax[1].errorbar(common_x, coef_df_yellow.mean(axis=0)[n_back:2*n_back],
                   yerr=[sem(coef_df_yellow.iloc[:, w + n_back].dropna()) for w in range(n_back)],
                   color='orange', marker='o', markersize=3, linestyle='-', 
                   lw=2, capsize=4, label='Foraging20B_GT')
    
    # Formatting
    for i in [0, 1]:
        ax[i].axhline(y=0, color='k', linestyle=':')
        ax[i].legend(frameon=False)
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['right'].set_visible(False)
    
    ax[0].set_ylim([-2, 6])
    ax[0].set_xlabel("Past Trials", fontsize=13)
    ax[0].set_ylabel(r"$\beta_{RewC(t-i)}$", fontsize=15)
    ax[0].set_title("RewC")
    ax[1].set_ylim([-25, 2])
    ax[1].set_xlabel("Past Trials", fontsize=13)
    ax[1].set_ylabel(r"$\beta_{UnrC(t-i)}$", fontsize=15)
    ax[1].set_title("UnrC")
    
    # Save plots
    plot_filename_png = os.path.join(save_directory, f'weights_plot_{env_name}_combined_{random_seed}.png')
    plot_filename_svg = os.path.join(save_directory, f'weights_plot_{env_name}_combined_{random_seed}.svg')
    
    plt.savefig(plot_filename_png, dpi=300, bbox_inches='tight')
    plt.savefig(plot_filename_svg, format='svg', bbox_inches='tight')
    print(f"Plots saved to:")
    print(f"PNG: {plot_filename_png}")
    print(f"SVG: {plot_filename_svg}")
    
    plt.close()  # Close the figure to free memory

# Run simulations for both environments
for env in Environments:
    run_simulations_for_environment(env)

print("\nAll simulations completed.") 